# Small GPT, little update

In [ ]:
#!pip install transformers
#!pip install huggingface_hub
#!pip install transformers
!pip install kaggle
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.5 MB/s eta 0:00:00


In [ ]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. GPU is ready for use.")
    print("Device Name:", torch.cuda.get_device_name(0))
    print("Number of GPUs available:", torch.cuda.device_count())
else:
    print("CUDA is not available. Running on CPU.")


CUDA is available. GPU is ready for use.
Device Name: NVIDIA A100-SXM4-40GB
Number of GPUs available: 1


In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ajazbi","key":"b05383cb636da48dedb1a00d1b53c555"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()


In [ ]:

import os
import time
import math
import pickle
from contextlib import nullcontext
import numpy as np
import pandas as pd
import tiktoken
from collections import Counter
import re
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group
from torch.utils.data import DataLoader, Dataset
import inspect
from sklearn.model_selection import train_test_split
import random
from transformers import AutoTokenizer
import threading
import queue


In [ ]:

# Authenticate with Kaggle
api = KaggleApi()
api.authenticate()

# Define Kaggle dataset details
dataset = 'aminghd/large-corpus-of-farsi-poems'
file_names = ['parvin_norm.txt','gilani_norm.txt','shahriar_norm.txt','khosro_norm.txt']
#file_names = ['parvin_norm.txt', 'eraghi_norm.txt', 'farrokhi_norm.txt',
#              'helali_norm.txt', 'gilani_norm.txt', 'khosro_norm.txt'
#              , 'salman_norm.txt', 'shahriar_norm.txt']  # Add more file names as needed

# Initialize variables to store the word count, max line length, and number of lines
total_words = 0
max_line_length = 0
num_lines = 0

# Initialize an empty list to store the formatted lines
formatted_lines = []

for file_name in file_names:
    # Download the specific file from the Kaggle dataset
    api.dataset_download_file(dataset, file_name)

    # Check if the file is downloaded and unzipped correctly
    if not os.path.exists(file_name):
        os.system(f'unzip {file_name}.zip')

    # Read the content of the text file
    with open(file_name, 'r', encoding='utf-8') as file:
        lines = [line.strip() for line in file.readlines() if line.strip()]

    # Iterate over the lines in pairs and format them with special tokens
    for i in range(0, len(lines), 2):
        if i + 1 < len(lines) and lines[i] and lines[i + 1]:
            formatted_line = f"[BOM] {lines[i]} [BOM] {lines[i+1]}[EOS]"
            formatted_lines.append(formatted_line)

            # Update line and word counts
            words_in_first_line = len(lines[i].split())
            words_in_second_line = len(lines[i + 1].split())

            total_words += words_in_first_line + words_in_second_line
            max_line_length = max(max_line_length, words_in_first_line, words_in_second_line)
            num_lines += 2  # Add 2 for each valid pair

# Join the formatted lines into a single string
sentences = "\n".join(formatted_lines)

# Print the entire formatted content
# print(sentences)

# Print the counts
print(f"Total words: {total_words}")
print(f"Max line length: {max_line_length} words")
print(f"Total number of lines: {num_lines}")


Dataset URL: https://www.kaggle.com/datasets/aminghd/large-corpus-of-farsi-poems
Dataset URL: https://www.kaggle.com/datasets/aminghd/large-corpus-of-farsi-poems
Dataset URL: https://www.kaggle.com/datasets/aminghd/large-corpus-of-farsi-poems
Dataset URL: https://www.kaggle.com/datasets/aminghd/large-corpus-of-farsi-poems
Total words: 447175
Max line length: 14 words
Total number of lines: 62794


In [ ]:
# Function to clean and count word frequencies
def count_word_frequencies(text):
    # Remove special tokens
    cleaned_text = re.sub(r'\[BOM\]|\[EOS\]', '', text)

    # Split text into words
    words = cleaned_text.split()

    # Count word frequencies
    word_counts = Counter(words)

    return word_counts

# Count word frequencies in the formatted content
word_frequencies = count_word_frequencies(sentences)

# Get the number of unique words
num_unique_words = len(word_frequencies)

# Sort word frequencies from highest to lowest
sorted_word_frequencies = word_frequencies.most_common()

# Print the total number of unique words
print(f'Total unique words: {num_unique_words}')

# Print word frequencies from highest to lowest
#for word, freq in sorted_word_frequencies:
#    print(f'{word}: {freq}')

Total unique words: 24786


In [ ]:
sentences=formatted_lines
# Check the number of sentences
num_sentences = len(sentences)
print(f"Number of sentences: {num_sentences}")

# Ensure there are enough sentences to split
if num_sentences > 1:
    # Calculate the split index
    split_index = int(num_sentences * 0.9)

    # Split data into training and validation sets (90% training, 10% validation)
    train_sentences = sentences[:split_index]
    val_sentences = sentences[split_index:]

    # Concatenate all training and validation sentences into single texts
    train_text = ' '.join(train_sentences)
    val_text = ' '.join(val_sentences)

    print(f"Number of training sentences: {len(train_sentences)}")
    print(f"Number of validation sentences: {len(val_sentences)}")
else:
    print("Not enough sentences to split. Please provide more data.")

# Load a Persian-specific tokenizer
tokenizer = AutoTokenizer.from_pretrained('bolbolzaban/gpt2-persian')
#tokenizer = AutoTokenizer.from_pretrained('HooshvareLab/gpt2-fa')

# Encode the data using the Persian-specific tokenizer
train_ids = tokenizer.encode(train_text, add_special_tokens=False)
val_ids = tokenizer.encode(val_text, add_special_tokens=False)

print(f"Training data has {len(train_ids):,} tokens")
print(f"Validation data has {len(val_ids):,} tokens")

# Convert to NumPy arrays and save to binary files  # <-- Changed to uint32
train_ids = np.array(train_ids, dtype=np.uint32)
val_ids = np.array(val_ids, dtype=np.uint32)


Number of sentences: 31397
Number of training sentences: 28257
Number of validation sentences: 3140


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/537k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

Training data has 528,268 tokens
Validation data has 54,897 tokens


In [ ]:
# Calculate total tokens
total_tokens = len(train_ids) + len(val_ids)

# Calculate unique tokens
unique_tokens = len(np.unique(np.concatenate((train_ids, val_ids))))

# Print total tokens and unique tokens
print(f"Total tokens: {total_tokens}")
print(f"Total unique tokens: {unique_tokens}")

# Calculate the length of tokens in each line
line_token_lengths = [len(tokenizer.encode(line, add_special_tokens=False)) for line in sentences]

# Find the maximum token length
max_token_length = max(line_token_lengths)

# Print the maximum token length
print(f"Maximum token length in a line: {max_token_length}")

Total tokens: 583165
Total unique tokens: 11331
Maximum token length in a line: 39


In [ ]:
# Print the vocabulary size
vocab_size = tokenizer.vocab_size
print(f"Tokenizer vocabulary size: {vocab_size}")

Tokenizer vocabulary size: 25000


In [ ]:
import math

def calculate_small_gpt_parameters(total_tokens, unique_tokens, compute_budget):
    """
    Calculate optimal GPT parameters for a small model based on modified Chinchilla scaling laws.

    Args:
    - total_tokens (int): Total number of tokens in the dataset.
    - unique_tokens (int): Total number of unique tokens (vocabulary size).
    - compute_budget (float): Total compute budget in FLOPs.

    Returns:
    - dict: Dictionary containing optimal GPT parameters.
    """
    # Modified Chinchilla scaling heuristic for token-to-parameter ratio (increased for smaller models)
    token_to_param_ratio = 40

    # Estimate the optimal model size
    optimal_model_size = math.sqrt(compute_budget / token_to_param_ratio)

    # Set tighter bounds for the parameters
    max_n_embd = 512   # Upper limit for embedding size in smaller models
    min_n_embd = 64    # Lower limit for embedding size in smaller models

    max_n_layer = 12   # Upper limit for layers in smaller models
    min_n_layer = 2    # Lower limit for layers in smaller models

    # Adjust the scaling to fit within the bounds
    scaling_factor = 1e10  # Increased to further reduce model size

    # Set the model parameters based on the optimal model size
    n_embd = int(min_n_embd + (optimal_model_size / scaling_factor) * (max_n_embd - min_n_embd))
    n_embd = max(min_n_embd, min(max_n_embd, n_embd))  # Ensure n_embd is within bounds

    n_layer = int(min_n_layer + (optimal_model_size / scaling_factor) * (max_n_layer - min_n_layer))
    n_layer = max(min_n_layer, min(max_n_layer, n_layer))  # Ensure n_layer is within bounds

    # Ensure n_embd is a multiple of n_head
    n_head = max(2, n_embd // 64)  # Ensure at least 1 head and n_embd is divisible by n_head
    n_embd = (n_embd // n_head) * n_head

    # fetch vocab_size for tokenizer
    vocab_size = tokenizer.vocab_size

    # Calculate approximate number of parameters
    num_params = n_embd * n_layer * (4 * n_embd + vocab_size) / 1e6  # in millions

    # Return the calculated parameters
    return {
        "n_embd": n_embd,
        "n_head": n_head,
        "n_layer": n_layer,
        "vocab_size": vocab_size,
        "block_size": 32,  # Reduced context size for smaller models
        "approx_params_millions": round(num_params, 2)
    }

# Example usage
compute_budget = 1e18  # Reduced compute budget for smaller models

gpt_parameters = calculate_small_gpt_parameters(total_tokens, unique_tokens, compute_budget)
print(gpt_parameters)

{'n_embd': 70, 'n_head': 2, 'n_layer': 2, 'vocab_size': 25000, 'block_size': 32, 'approx_params_millions': 3.54}


In [ ]:
sample_size=64
print("Training Data Sample:")
print(tokenizer.decode(train_ids[:sample_size]))

print("\nValidation Data Sample:")
print(tokenizer.decode(val_ids[:sample_size]))



Training Data Sample:
[BOM] ای دل عبث مخور غم دنیا را[BOM] فکرت مکن نیامده فردا را[EOS][BOM] کنج قفس چو نیک بیندیشی[BOM] چون گلشن است مرغ شکیبا را[EOS][BOM] بشکاف خاک را و ببین آنگه[BOM] بی مهری زمانه رسوا را[EOS][BOM] این دشت خوابگاه شهیدانست[BOM] فرصت شمار وقت تماشا را[EOS][BOM] از عمر رفته نیز شماری کن

Validation Data Sample:
[BOM] عشق از دو صنم بود عنان تاب[BOM] چون دین ز توجه دو محراب[EOS][BOM] تا یک سر مو بود به جایت[BOM] یک مو نکشم سر از هوایت[EOS][BOM] اینجا من و دلستانم آنجاست[BOM] آنجاست دلم که جانم آنجاست[EOS][BOM] گر کرد سپهر بی طریقم[BOM] تهمت زده دگر رفیقم[EOS][BOM]


In [ ]:

class CustomDataset(Dataset):
    def __init__(self, data, block_size):
        self.data = data
        self.block_size = block_size

    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # Randomly sample starting index
        start_idx = np.random.randint(0, len(self.data) - self.block_size - 1)
        x = self.data[start_idx:start_idx + self.block_size]
        y = self.data[start_idx + 1:start_idx + self.block_size + 1]
        return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)


def get_data_loader(data, batch_size, block_size):
    dataset = CustomDataset(data, block_size)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

batch_size = 256
block_size = 64

# Create DataLoaders for training and validation data
train_loader = get_data_loader(train_ids, batch_size, block_size)
val_loader = get_data_loader(val_ids, batch_size, block_size)


In [ ]:
def print_first_and_last_five(data_loader, tokenizer):
    """
    Print the first and last five data points (x and y) from the DataLoader.

    Args:
    - data_loader (DataLoader): The DataLoader object.
    - tokenizer (AutoTokenizer): The tokenizer for decoding.
    """
    x_batch, y_batch = next(iter(data_loader))

    def print_data_points(data, label):
        batch_size = data.size(0)
        seq_len = data.size(1)

        print(f"\nFirst 5 sequences in {label}:")
        for i in range(min(5, batch_size)):
            tokens = data[i].tolist()
            decoded = tokenizer.decode(tokens)
            print(f"{label}[{i}]: Tokens: {tokens}, Decoded: {decoded}")

        print(f"\nLast 5 sequences in {label}:")
        for i in range(max(0, batch_size - 5), batch_size):
            tokens = data[i].tolist()
            decoded = tokenizer.decode(tokens)
            print(f"{label}[{i}]: Tokens: {tokens}, Decoded: {decoded}")

    print("First and Last 5 data points in x:")
    print_data_points(x_batch, 'x')

    print("\nFirst and Last 5 data points in y:")
    print_data_points(y_batch, 'y')


# Print the first and last 5 sequences of both x and y for training and validation data
print("Training Data:")
print_first_and_last_five(train_loader, tokenizer)

print("\nValidation Data:")
print_first_and_last_five(val_loader, tokenizer)


Training Data:
First and Last 5 data points in x:

First 5 sequences in x:
x[0]: Tokens: [107, 7356, 48, 542, 7, 2298, 291, 1091, 1984, 45, 769, 80, 605, 2960, 61, 161, 213, 9, 7, 213, 562, 22812, 612, 10681, 1545, 99, 213, 7, 96, 209, 48, 73, 681, 127, 991, 15920, 46, 209, 99, 213, 9, 7, 46, 19834, 72, 90, 3229, 129, 423, 52, 836, 889, 7, 21001, 51, 46, 90, 1670, 213, 606, 128, 3611, 99, 213], Decoded: نمی مانی به کس[BOM] خسرو همان بنده ست و بس تو آنکه بودی آن نه ای[EOS][BOM] ای درد بیدرد دلم تاراج پنهان کرده ای[BOM] یا جان بهم بیرون روی کآرام در جان کرده ای[EOS][BOM] در حیرتم تا هر شبی چون خواب می آید ترا[BOM] زینسان که در هر گوشه ای صد دل پریشان کرده ای
x[1]: Tokens: [51, 105, 1160, 120, 542, 137, 6538, 9, 7, 573, 61, 10984, 51, 6293, 19507, 7, 2910, 57, 7846, 67, 15415, 157, 9, 7, 2792, 94, 51, 2177, 50, 116, 10663, 7, 2177, 53, 95, 2171, 864, 48, 131, 9, 7, 1376, 57, 90, 51, 9212, 92, 7625, 7, 90, 116, 53, 2918, 4121, 668, 11102, 9, 7, 1091, 2879, 1669, 121, 7, 333], Decoded: که ز

In [ ]:
# Custom Dataset class
class GPTDataset(Dataset):
    def __init__(self, data, block_size):
        self.data = data
        self.block_size = block_size
        # Ensure the data is a torch tensor
        if not isinstance(self.data, torch.Tensor):
            self.data = torch.tensor(self.data, dtype=torch.long)

    def __len__(self):
        return len(self.data) - self.block_size - 1

    def __getitem__(self, idx):
        if idx >= len(self):
            raise IndexError("Index out of range")
        x = self.data[idx:idx + self.block_size]
        y = self.data[idx + 1:idx + 1 + self.block_size]
        return x, y

class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        # regularization
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.dropout = config.dropout
        # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:
            # manual implementation of attention
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu    = nn.GELU()
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = LayerNorm(config.n_embd, bias=config.bias),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.transformer.wte.weight = self.lm_head.weight  # Weight tying

        # Init all weights
        self.apply(self._init_weights)
        # Apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02 / math.sqrt(2 * config.n_layer))

        # Report number of parameters
        print("number of parameters: %.2fM" % (self.get_num_params() / 1e6,))

    def get_num_params(self, non_embedding=True):
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device)  # shape (t)

        # Forward the GPT model itself
        tok_emb = self.transformer.wte(idx)  # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos)  # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # If we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # Inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :])  # note: using list [-1] to preserve the time dim
            loss = None

        return logits, loss

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        # start with all of the candidate parameters
        param_dict = {pn: p for pn, p in self.named_parameters()}
        # filter out those that do not require grad
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"using fused AdamW: {use_fused}")

        return optimizer


In [ ]:
# Initialize the model
from dataclasses import dataclass

@dataclass
class GPTConfig:
    block_size: int
    vocab_size: int
    n_layer: int
    n_head: int
    n_embd: int  # n_embd must be divisible by n_head
    dropout: float
    bias: bool
"""
config = GPTConfig(
    block_size=64,
    vocab_size= vocab_size,
    n_layer=4,
    n_head=4,
    n_embd=64,  # Ensure n_embd is divisible by n_head
    dropout=0.1,
    bias=True
)
"""

config = GPTConfig(
    block_size=block_size,
    vocab_size= vocab_size,
    n_layer=4,
    n_head=4,
    n_embd=128,  # Ensure n_embd is divisible by n_head
    dropout=0.1,
    bias=True
)


model = GPT(config)
print(model)

print ("vocab_size=" , vocab_size)


number of parameters: 3.99M
GPT(
  (transformer): ModuleDict(
    (wte): Embedding(25000, 128)
    (wpe): Embedding(64, 128)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-3): 4 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=128, out_features=384, bias=True)
          (c_proj): Linear(in_features=128, out_features=128, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=128, out_features=512, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=512, out_features=128, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=128, out_features=25000, bias=False)
)
vocab_size= 25000


In [ ]:

# Initialize optimizer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)  # Ensure the model is on the correct device
optimizer = model.configure_optimizers(weight_decay=0.1, learning_rate=3e-4, betas=(0.9, 0.95), device_type=device)

num decayed parameter tensors: 18, with 3,994,624 parameters
num non-decayed parameter tensors: 34, with 6,912 parameters
using fused AdamW: True


In [ ]:
# Define the learning rate scheduler
def get_lr(it):
    # 1) Linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) If it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) In between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))  # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)


# Function to handle user input in a separate thread
def check_for_stop(stop_queue):
    while True:
        user_input = input()
        if user_input.lower() == 'q':
            stop_queue.put(True)
            break


In [ ]:
# Create a queue to communicate between the input thread and the main thread
stop_queue = queue.Queue()

# Start the input thread
input_thread = threading.Thread(target=check_for_stop, args=(stop_queue,))
input_thread.daemon = True
input_thread.start()

# Define global configurations and hyperparameters

config = {
    'learning_rate': 3e-4,
    'always_save_checkpoint': True,
    'out_dir': './checkpoints',
    'eval_only': False,
    'gradient_accumulation_steps': 4,
    'eval_interval': 200,
    'max_iters': 20000,
    'log_interval': 10,
    'grad_clip': 0.1,
    'decay_lr': False,
    'max_epochs': 50,
    'target_val_loss': 2  # Example target validation loss
}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Disable scaler and autocast if CUDA is not available
use_amp = torch.cuda.is_available()
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

model.to(device)
model.train()
iter_num = 0
best_val_loss = float('inf')
epoch = 0

while epoch < config['max_epochs']:
    train_loader = get_data_loader(train_ids, batch_size, block_size)
    val_loader = get_data_loader(val_ids, batch_size, block_size)

    for batch_idx, (X, Y) in enumerate(train_loader):
        X, Y = X.to(device), Y.to(device)

        # Determine and set the learning rate for this iteration
        lr = get_lr(iter_num) if config['decay_lr'] else config['learning_rate']
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        # Forward, backward update with gradient accumulation
        for micro_step in range(config['gradient_accumulation_steps']):
            logits, loss = model(X, Y)
            loss = loss / config['gradient_accumulation_steps']  # scale the loss to account for gradient accumulation
            scaler.scale(loss).backward()

        # Clip the gradient
        if config['grad_clip'] != 0.0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), config['grad_clip'])
        # Step the optimizer and scaler
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad(set_to_none=True)

        iter_num += 1

        # Print loss every 50 iterations
        if iter_num % 50 == 0:
            print(f"Iteration {iter_num}, Loss: {loss.item()}")


        # Evaluate the loss on train/val sets and write checkpoints
        if iter_num % config['eval_interval'] == 0:
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for val_X, val_Y in val_loader:
                    val_X, val_Y = val_X.to(device), val_Y.to(device)
                    logits, loss = model(val_X, val_Y)
                    val_loss += loss.item()
            val_loss /= len(val_loader)
            model.train()

            print(f"step {iter_num}: val loss {val_loss:.4f}")
            if val_loss < best_val_loss or config['always_save_checkpoint']:
                best_val_loss = val_loss
                if iter_num > 0:
                    checkpoint = {
                        'model': model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'config': config,
                        'iter_num': iter_num,
                        'best_val_loss': best_val_loss,
                    }
                    #print(f"saving checkpoint to {config['out_dir']}")
                    #torch.save(checkpoint, os.path.join(config['out_dir'], 'ckpt.pt'))
            if val_loss < config['target_val_loss']:
                print("Target validation loss achieved. Stopping training.")
                break

        # Termination condition
        if iter_num >= config['max_iters']:
            break

    if not stop_queue.empty():
        break

    epoch += 1
    if iter_num >= config['max_iters']:
        break


Iteration 50, Loss: 1.9202301502227783
Iteration 100, Loss: 1.6358609199523926
Iteration 150, Loss: 1.542484998703003
Iteration 200, Loss: 1.4966806173324585
step 200: val loss 5.9998
Iteration 250, Loss: 1.4721651077270508
Iteration 300, Loss: 1.4386447668075562
Iteration 350, Loss: 1.4046943187713623
Iteration 400, Loss: 1.3978220224380493
step 400: val loss 5.6523
Iteration 450, Loss: 1.3835545778274536
Iteration 500, Loss: 1.3691529035568237
Iteration 550, Loss: 1.3690887689590454
Iteration 600, Loss: 1.359129786491394
step 600: val loss 5.5089
Iteration 650, Loss: 1.3417115211486816
Iteration 700, Loss: 1.334824800491333
Iteration 750, Loss: 1.3214160203933716
Iteration 800, Loss: 1.3144093751907349
step 800: val loss 5.4175
Iteration 850, Loss: 1.304896593093872
Iteration 900, Loss: 1.3021924495697021
Iteration 950, Loss: 1.2948496341705322
Iteration 1000, Loss: 1.293408751487732
step 1000: val loss 5.3308
Iteration 1050, Loss: 1.2786697149276733
Iteration 1100, Loss: 1.267081260

KeyboardInterrupt: 

In [ ]:
# Save the trained model, optimizer state, and configuration details
import json

# Save the trained model, optimizer state, and configuration details
save_dir = 'GPTper_saves'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Save the model state
torch.save(model.state_dict(), os.path.join(save_dir, 'final_model_perchin.pt'))

# Save the optimizer state
torch.save(optimizer.state_dict(), os.path.join(save_dir, 'final_optimizer_perchin.pt'))

# Save the configuration
config_path = os.path.join(save_dir, 'config_perchin.json')
with open(config_path, 'w') as config_file:
    json.dump(config, config_file, indent=4)

print(f'Model, optimizer state, and configuration saved to {save_dir}')


Model, optimizer state, and configuration saved to GPTper_saves


In [ ]:
# Function to tokenize input text and truncate to block size
def tokenize_and_truncate_text(text, block_size):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    if len(tokens) > block_size:
        tokens = tokens[-block_size:]  # Truncate to the last block_size tokens
    return torch.tensor(tokens, dtype=torch.long).unsqueeze(0)  # Add batch dimension

# Function to decode output tokens
def decode_output(predicted_ids):
    predicted_tokens = predicted_ids.tolist()  # Convert tensor to list
    predicted_text = tokenizer.decode(predicted_tokens)
    return predicted_text

# Function to get model output for a single input text
def get_model_output(model, input_text, config, temperature=1.0, top_k=50):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Tokenize and truncate input text
    input_ids = tokenize_and_truncate_text(input_text, block_size)
    input_ids = input_ids.to(device)
    model.to(device)

    with torch.no_grad():
        logits, _ = model(input_ids)
        logits = logits[:, -1, :] / temperature  # Apply temperature

        # Top-k sampling
        top_k_logits, top_k_indices = torch.topk(logits, k=top_k, dim=-1)
        probs = F.softmax(top_k_logits, dim=-1)

        # Sample from the distribution
        chosen_idx = torch.multinomial(probs, num_samples=1)
        predicted_token_id = top_k_indices[0, chosen_idx[0]]

        # Decode the predicted token
        predicted_text = decode_output(predicted_token_id)

    return predicted_text

# Function to validate the predicted text
def is_valid_token(token):
    # Implement your logic to check if the token is valid or not
    # This can include checking for known invalid sequences, punctuation, etc.
    invalid_sequences = ['ا ی']
    return token not in invalid_sequences

# Sample input text
input_text = "بنشین که با من هر نظر"
input_text = "حکیمان پیشین چنین گفته اند"
input_text = "با نصرت و فتح و ظفر و دولت والا"
input_text = "توانا بود هر که دانا بود"
input_text = "تو خوشگل ناز منی "
input_text = "ای دوست"
input_text = "ای صبا"
input_text = "دور است کاروان سحر"
input_text = "از عشق تو من مردم "
input_text = "تیر نگاه تو در دل من "




# Define maximum BOM count
max_bom_count = 10

# Accumulate the complete generated text
complete_text = input_text

# Track the last few generated tokens
repetition_threshold = 5
last_few_tokens = []
temperature = 0.7

# Generate text in a loop until the maximum BOM count is reached
bom_count = 0

while bom_count < max_bom_count:
    predicted_text = get_model_output(model, input_text, config, temperature=0.7, top_k=50)

    # Prevent repeating sequences and filter invalid tokens
    if predicted_text.strip() in last_few_tokens or not is_valid_token(predicted_text.strip()):
        temperature *= 1.1  # Increase temperature to promote diversity
        predicted_text = get_model_output(model, input_text, config, temperature=temperature, top_k=50)
    else:
        temperature = max(0.7, temperature * 0.95)  # Gradually decrease temperature back to a minimum

    complete_text += "‌‌‌‌‌ " + predicted_text.strip()  # Ensure proper spacing
    last_few_tokens.append(predicted_text.strip())

    if len(last_few_tokens) > repetition_threshold:
        last_few_tokens.pop(0)

    # Truncate the input text if it exceeds the block size
    input_text += " " + predicted_text.strip()
    input_ids = tokenize_and_truncate_text(input_text, block_size)
    input_text = tokenizer.decode(input_ids[0].tolist())  # Update input_text to truncated version

    # Add a new line after [EOS]
    if '[EOS]' and '[BOM]' in predicted_text:
        complete_text += '\n'
        bom_count += 1

# Print the whole output after the loop
print(f"Predicted Text:\n{complete_text}\n")


Predicted Text:
تیر نگاه تو در دل من ‌‌‌‌‌ مسکین‌‌‌‌‌ [BOM]
‌‌‌‌‌ چه‌‌‌‌‌ دانی‌‌‌‌‌ که‌‌‌‌‌ به‌‌‌‌‌ یک‌‌‌‌‌ نظاره‌‌‌‌‌ تو‌‌‌‌‌ داری‌‌‌‌‌ [EOS]‌‌‌‌‌ [BOM]
‌‌‌‌‌ به‌‌‌‌‌ هر‌‌‌‌‌ ذره‌‌‌‌‌ ای‌‌‌‌‌ که‌‌‌‌‌ مرا‌‌‌‌‌ در‌‌‌‌‌ هوس‌‌‌‌‌ روی‌‌‌‌‌ تو‌‌‌‌‌ [BOM]
‌‌‌‌‌ به‌‌‌‌‌ روز‌‌‌‌‌ خویش‌‌‌‌‌ می‌‌‌‌‌ کشم‌‌‌‌‌ و‌‌‌‌‌ شب‌‌‌‌‌ در‌‌‌‌‌ همه‌‌‌‌‌ کس‌‌‌‌‌ بودی‌‌‌‌‌ [EOS]‌‌‌‌‌ [BOM]
‌‌‌‌‌ چو‌‌‌‌‌ جان‌‌‌‌‌ دهم‌‌‌‌‌ سخن‌‌‌‌‌ از‌‌‌‌‌ دیدن‌‌‌‌‌ تو‌‌‌‌‌ [BOM]
‌‌‌‌‌ به‌‌‌‌‌ جان‌‌‌‌‌ دهم‌‌‌‌‌ و‌‌‌‌‌ آن‌‌‌‌‌ قد‌‌‌‌‌ او‌‌‌‌‌ که‌‌‌‌‌ داری‌‌‌‌‌ [EOS]‌‌‌‌‌ [BOM]
‌‌‌‌‌ به‌‌‌‌‌ دل‌‌‌‌‌ رسید‌‌‌‌‌ هجر‌‌‌‌‌ تو‌‌‌‌‌ هر‌‌‌‌‌ دم‌‌‌‌‌ ار‌‌‌‌‌ چه‌‌‌‌‌ هست‌‌‌‌‌ [BOM]
‌‌‌‌‌ چو‌‌‌‌‌ نیست‌‌‌‌‌ امید‌‌‌‌‌ زیستن‌‌‌‌‌ بی‌‌‌‌‌ جای‌‌‌‌‌ ما‌‌‌‌‌ بودی‌‌‌‌‌ [EOS]‌‌‌‌‌ [BOM]
‌‌‌‌‌ دل‌‌‌‌‌ از‌‌‌‌‌ تو‌‌‌‌‌ بسوخت‌‌‌‌‌ م‌‌‌‌‌ سوز‌‌‌‌‌ و‌‌‌‌‌ گداز‌‌‌‌‌ نتوان‌‌‌‌‌ کرد‌‌‌‌‌ [BOM]
‌‌‌‌‌ هنوز‌‌‌‌‌ دلی‌‌‌‌‌ ز‌‌‌‌‌ تو‌‌‌‌‌ خام‌‌‌‌‌ باشد‌‌‌‌‌ [EOS]‌‌‌‌‌ [BOM]


